In [66]:
import numpy as np
from Druckrohrleitung_class_file import Druckrohrleitung_class
import matplotlib.pyplot as plt

#importing pressure conversion function
import sys
import os
current = os.path.dirname(os.path.realpath('Main_Programm.ipynb'))
parent = os.path.dirname(current)
sys.path.append(parent)
from functions.pressure_conversion import pressure_conversion
from Ausgleichsbecken.Ausgleichsbecken_class_file import Ausgleichsbecken_class

In [67]:
# define constants

    # for physics
g                   = 9.81                                          # [m/s²]    gravitational acceleration 
rho                 = 1000.                                         # [kg/m³]   density of water 
pUnit_calc          = 'Pa'                                          # [text]    DO NOT CHANGE! for pressure conversion in print statements and plot labels 
pUnit_conv          = 'mWS'                                         # [text]    for pressure conversion in print statements and plot labels

    # for Turbine
Tur_Q_nenn          = 0.85                                          # [m³/s]    nominal flux of turbine 
Tur_p_nenn          = pressure_conversion(10.6,'bar',pUnit_calc)    # [Pa]      nominal pressure of turbine 
Tur_closingTime     = 90.                                           # [s]       closing time of turbine

    # for PI controller
Con_targetLevel     = 8.                                            # [m]
Con_K_p             = 0.1                                           # [-]       proportional constant of PI controller
Con_T_i             = 10.                                           # [s]       timespan in which a steady state error is corrected by the intergal term
Con_deadbandRange   = 0.05                                          # [m]       Deadband range around targetLevel for which the controller does NOT intervene

    # for pipeline
Pip_length          = (535.+478.)                                   # [m]       length of pipeline
Pip_dia             = 0.9                                           # [m]       diameter of pipeline
Pip_area            = Pip_dia**2/4*np.pi                            # [m²]      crossectional area of pipeline
Pip_head            = 105.                                          # [m]       hydraulic head of pipeline without reservoir
Pip_angle           = np.arcsin(Pip_head/Pip_length)                # [rad]     elevation angle of pipeline 
Pip_n_seg           = 50                                            # [-]       number of pipe segments in discretization
Pip_f_D             = 0.014                                         # [-]       Darcy friction factor
Pip_pw_vel          = 500.                                          # [m/s]     propagation velocity of the pressure wave (pw) in the given pipeline
    # derivatives of the pipeline constants
Pip_dx              = Pip_length/Pip_n_seg                          # [m]       length of each pipe segment
Pip_dt              = Pip_dx/Pip_pw_vel                             # [s]       timestep according to method of characteristics
Pip_nn              = Pip_n_seg+1                                   # [1]       number of nodes
Pip_x_vec           = np.arange(0,Pip_nn,1)*Pip_dx                  # [m]       vector holding the distance of each node from the upstream reservoir along the pipeline
Pip_h_vec           = np.arange(0,Pip_nn,1)*Pip_head/Pip_n_seg      # [m]       vector holding the vertival distance of each node from the upstream reservoir

    # for reservoir
Res_area_base       = 74.                                           # [m²]      total base are of the cuboid reservoir   
Res_area_out        = Pip_area                                      # [m²]      outflux area of the reservoir, given by pipeline area
Res_level_crit_lo   = 0.                                            # [m]       for yet-to-be-implemented warnings
Res_level_crit_hi   = np.inf                                        # [m]       for yet-to-be-implemented warnings
Res_dt_approx       = 1e-3                                          # [s]       approx. timestep of reservoir time evolution to ensure numerical stability (see Res_nt why approx.)
Res_nt              = max(1,int(Pip_dt//Res_dt_approx))             # [1]       number of timesteps of the reservoir time evolution within one timestep of the pipeline
Res_dt              = Pip_dt/Res_nt                                 # [s]       harmonised timestep of reservoir time evolution

    # for general simulation
flux_init           = Tur_Q_nenn/1.1                                # [m³/s]    initial flux through whole system for steady state initialization  
level_init          = Con_targetLevel                               # [m]       initial water level in upstream reservoir for steady state initialization
simTime_target      = 1200.                                         # [s]       target for total simulation time (will vary slightly to fit with Pip_dt)
nt                  = int(simTime_target//Pip_dt)                   # [1]       Number of timesteps of the whole system
t_vec               = np.arange(0,nt+1,1)*Pip_dt                    # [s]       time vector. At each step of t_vec the system parameters are stored


In [68]:
# create objects

# Upstream reservoir
reservoir = Ausgleichsbecken_class(Res_area_base,Res_area_out,Res_dt,pUnit_conv,Res_level_crit_lo,Res_level_crit_hi,rho)
reservoir.set_steady_state(flux_init,level_init)

# pipeline
pipe = Druckrohrleitung_class(Pip_length,Pip_dia,Pip_head,Pip_n_seg,Pip_f_D,Pip_pw_vel,Pip_dt,pUnit_conv,rho)
pipe.set_steady_state(flux_init,reservoir.get_current_pressure())
pipe.get_info()

p_0 = pipe.get_initial_pressure_distribution()

The pipeline has the following attributes: 
----------------------------- 
Length                =       1013.0     m 
Diameter              =       0.9        m 
Hydraulic head        =       105.0      m 
Number of segments    =       50         
Number of nodes       =       51         
Length per segments   =       20.26      m 
Pipeline angle        =       0.104      rad 
Pipeline angle        =       5.95° 
Darcy friction factor =       0.014      
Density of liquid     =       1000.0     kg/m³ 
Pressure wave vel.    =       500.0      m/s 
Simulation timestep   =       0.04052    s 
----------------------------- 
Velocity and pressure distribution are vectors and are accessible via the 
                  get_current_velocity_distribution() and get_current_pressure_distribution() methods of the pipeline object. 
                     See also get_lowest_XXX_per_node() and get_highest_XXX_per_node() methods.


In [69]:
# initialization for timeloop

level_vec       = np.zeros_like(t_vec)
level_vec[0]    = reservoir.get_current_level()
volume_vec      = np.zeros_like(t_vec)                               
volume_vec[0]   = reservoir.get_current_volume()


# prepare the vectors in which the pressure and velocity distribution in the pipeline from the previous timestep are stored
v_old = pipe.get_current_velocity_distribution()
Q_old = pipe.get_current_flux_distribution()
p_old = pipe.get_current_pressure_distribution()

# prepare the vectors in which the temporal evolution of the boundary conditions are stored
    # keep in mind, that the velocity at the turbine and the pressure at the reservoir are set manually and
        # through the time evolution of the reservoir respectively  
    # the pressure at the turbine and the velocity at the reservoir are calculated from the method of characteristics
v_boundary_res  = np.zeros_like(t_vec)
v_boundary_tur  = np.full_like(t_vec,v_old[-1])
p_boundary_res  = np.zeros_like(t_vec)
p_boundary_tur  = np.zeros_like(t_vec)
Q_boundary_res  = np.zeros_like(t_vec)
Q_boundary_tur  = np.zeros_like(t_vec)

# set the boundary conditions for the first timestep
v_boundary_res[0]   = v_old[0]
p_boundary_res[0]   = p_old[0]
Q_boundary_res[0]   = Q_old[0]
p_boundary_tur[0]   = p_old[-1]
Q_boundary_tur[0]   = Q_old[-1]


In [70]:
%matplotlib qt5
# create a figure and subplots to display the velocity and pressure distribution across the pipeline in each pipeline step
fig1,axs1 = plt.subplots(3,1)
fig1.suptitle(str(0) +' s / '+str(round(t_vec[-1],2)) + ' s' )
axs1[0].set_title('Pressure distribution in pipeline')
axs1[0].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[0].set_ylabel(r'$p$ ['+pUnit_conv+']')
axs1[0].set_ylim([-2,Pip_head*1.1])
axs1[1].set_title('Pressure distribution in pipeline \n Difference to t=0')
axs1[1].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[1].set_ylabel(r'$p$ ['+pUnit_conv+']')
axs1[2].set_title('Flux distribution in pipeline')
axs1[2].set_xlabel(r'$x$ [$\mathrm{m}$]')
axs1[2].set_ylabel(r'$Q$ [$\mathrm{m}^3 / \mathrm{s}$]')
# create line objects (lo) whoes values can be updated in time loop to animate the evolution
lo_0,       = axs1[0].plot(Pip_x_vec,pressure_conversion(p_old,pUnit_calc, pUnit_conv),marker='.')
lo_0min,    = axs1[0].plot(Pip_x_vec,pressure_conversion(pipe.get_lowest_pressure_per_node(),pUnit_calc,pUnit_conv),c='red')
lo_0max,    = axs1[0].plot(Pip_x_vec,pressure_conversion(pipe.get_highest_pressure_per_node(),pUnit_calc,pUnit_conv),c='red')
lo_1,       = axs1[1].plot(Pip_x_vec,pressure_conversion(p_old-p_0,pUnit_calc, pUnit_conv),marker='.')
lo_1min,    = axs1[1].plot(Pip_x_vec,pressure_conversion(pipe.get_lowest_pressure_per_node()-p_0,pUnit_calc,pUnit_conv),c='red')
lo_1max,    = axs1[1].plot(Pip_x_vec,pressure_conversion(pipe.get_highest_pressure_per_node()-p_0,pUnit_calc,pUnit_conv),c='red')
lo_2,       = axs1[1].plot(Pip_x_vec,Q_old,marker='.')
lo_2min,    = axs1[2].plot(Pip_x_vec,pipe.get_lowest_flux_per_node(),c='red')
lo_2max,    = axs1[2].plot(Pip_x_vec,pipe.get_highest_flux_per_node(),c='red')

# axs1[0].autoscale()
# axs1[1].autoscale()

fig1.tight_layout()
fig1.show()

In [71]:
for it_pipe in range(1,nt+1):
# for each pipeline timestep, execute nt_eRK4 timesteps of the reservoir code
    # set initial conditions for the reservoir time evolution calculted with e-RK4
    reservoir.set_pressure(p_old[0],display_warning=False)
    reservoir.set_outflux(v_old[0]*Pip_area,display_warning=False)
    # calculate the time evolution of the reservoir level within each pipeline timestep to avoid runaway numerical error
    for it_res in range(Res_nt):
        reservoir.timestep_reservoir_evolution()                                                             
    level_vec[it_pipe] = reservoir.get_current_level()
    volume_vec[it_pipe] = reservoir.get_current_volume()         
    
    # set boundary conditions for the next timestep of the characteristic method
    p_boundary_res[it_pipe] = reservoir.get_current_pressure()
    # v_boundary_tur[it_pipe] = flux_init/Pip_area

    # the the boundary conditions in the pipe.object and thereby calculate boundary pressure at turbine
    pipe.set_boundary_conditions_next_timestep(p_boundary_res[it_pipe],v_boundary_tur[it_pipe])
    p_boundary_tur[it_pipe] = pipe.get_current_pressure_distribution()[-1]
    v_boundary_res[it_pipe] = pipe.get_current_velocity_distribution()[0]
    Q_boundary_res[it_pipe] = pipe.get_current_flux_distribution()[0]
    Q_boundary_tur[it_pipe] = pipe.get_current_flux_distribution()[-1]

    # perform the next timestep via the characteristic method
    pipe.timestep_characteristic_method_vectorized()

    # prepare for next loop
    p_old = pipe.get_current_pressure_distribution()
    v_old = pipe.get_current_velocity_distribution()

    # plot some stuff
        # remove line-objects to autoscale axes (there is definetly a better way, but this works ¯\_(ツ)_/¯ )
    if it_pipe%50 == 0:     # only plot every 50th iteration for performance reasons (plotting takes the most amount of time)
        lo_0.remove()
        lo_0min.remove()
        lo_0max.remove()
        lo_1.remove()
        lo_1min.remove()
        lo_1max.remove()
        lo_2.remove()
        lo_2min.remove()
        lo_2max.remove()
            # plot new pressure and velocity distribution in the pipeline
        lo_0, = axs1[0].plot(Pip_x_vec,pressure_conversion(pipe.get_current_pressure_distribution(),pUnit_calc,pUnit_conv),marker='.',c='blue')
        lo_0min, = axs1[0].plot(Pip_x_vec,pressure_conversion(pipe.get_lowest_pressure_per_node(),pUnit_calc,pUnit_conv),c='red')
        lo_0max, = axs1[0].plot(Pip_x_vec,pressure_conversion(pipe.get_highest_pressure_per_node(),pUnit_calc,pUnit_conv),c='red')        
        lo_1, = axs1[1].plot(Pip_x_vec,pressure_conversion(pipe.get_current_pressure_distribution()-p_0,pUnit_calc,pUnit_conv),marker='.',c='blue')
        lo_1min, = axs1[1].plot(Pip_x_vec,pressure_conversion(pipe.get_lowest_pressure_per_node()-p_0,pUnit_calc,pUnit_conv),c='red')
        lo_1max, = axs1[1].plot(Pip_x_vec,pressure_conversion(pipe.get_highest_pressure_per_node()-p_0,pUnit_calc,pUnit_conv),c='red')
        lo_2, = axs1[2].plot(Pip_x_vec,pipe.get_current_flux_distribution(),marker='.',c='blue')
        lo_2min, = axs1[2].plot(Pip_x_vec,pipe.get_lowest_flux_per_node(),c='red')
        lo_2max, = axs1[2].plot(Pip_x_vec,pipe.get_highest_flux_per_node(),c='red')
        fig1.suptitle(str(round(t_vec[it_pipe],2))+ ' s / '+str(round(t_vec[-1],2)) + ' s' )
        fig1.canvas.draw() # force figure output
        fig1.tight_layout()
        fig1.show()
        plt.pause(0.1)  

reservoir.get_info(full=True)
pipe.get_info()

The cuboid reservoir has the following attributes: 
----------------------------- 
Base area             =       74.0       m² 
Outflux area          =       0.636      m² 
Current level         =       8.0        m
Critical level low    =       0.0        m 
Critical level high   =       inf        m 
Volume in reservoir   =       592.0      m³ 
Current influx        =       0.773      m³/s 
Current outflux       =       0.773      m³/s 
Current outflux vel   =       1.215      m/s 
Current pipe pressure =       7.854      mWS 
Simulation timestep   =       0.001013   s 
Density of liquid     =       1000.0     kg/m³ 
----------------------------- 

The pipeline has the following attributes: 
----------------------------- 
Length                =       1013.0     m 
Diameter              =       0.9        m 
Hydraulic head        =       105.0      m 
Number of segments    =       50         
Number of nodes       =       51         
Length per segments   =       20.26      m 
Pipeli

In [73]:
level_plot_min = 0
level_plot_max = 15

fig3,axs3 = plt.subplots(2,2,figsize=(16,9))
fig3.suptitle('Fläche = '+str(Res_area_base)+'\n'+'Kp = '+str(Con_K_p)+'  Ti = '+str(Con_T_i))
axs3[0,0].set_title('Level and Volume reservoir')
axs3[0,0].plot(t_vec,level_vec,label='level')
axs3[0,0].plot(t_vec,np.full_like(t_vec,Res_level_crit_lo),label='level_limit',c='r')
axs3[0,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[0,0].set_ylabel(r'$h$ [m]')
axs3[0,0].set_ylim(level_plot_min,level_plot_max)
x_twin_00 = axs3[0,0].twinx()
x_twin_00.set_ylabel(r'$V$ [$\mathrm{m}^3$]')
x_twin_00.plot(t_vec,volume_vec)
x_twin_00.set_ylim(volume_plot_min,volume_plot_max)
axs3[0,0].legend()

# axs3[0,1].set_title('LA')
# axs3[0,1].plot(t_vec,100*OL_T1_LA_soll_vec,label='OL_T1 Target',c='b')
# axs3[0,1].scatter(t_vec[::200],100*OL_T1_LA_ist_vec[::200],label='OL_T1 Actual',c='b',marker='+')
# axs3[0,1].plot(t_vec,100*OL_T2_LA_soll_vec,label='OL_T2 Target',c='g')
# axs3[0,1].scatter(t_vec[::200],100*OL_T2_LA_ist_vec[::200],label='OL_T2 Actual',c='g',marker='+')
# axs3[0,1].plot(t_vec,100*UL_T1_LA_soll_vec,label='UL_T1 Target',c='r')
# axs3[0,1].scatter(t_vec[::200],100*UL_T1_LA_ist_vec[::200],label='UL_T1 Actual',c='r',marker='+')
# axs3[0,1].plot(t_vec,100*UL_T2_LA_soll_vec,label='UL_T2 Target',c='k')
# axs3[0,1].scatter(t_vec[::200],100*UL_T2_LA_ist_vec[::200],label='UL_T2 Actual',c='k',marker='+')
# axs3[0,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
# axs3[0,1].set_ylabel(r'$LA$ [%]')
# axs3[0,1].legend()

axs3[1,0].set_title('Fluxes')
axs3[1,0].plot(t_vec,np.full_like(t_vec,flux_init),label='Influx')
axs3[1,0].plot(t_vec,Q_boundary_res,label='Outflux')
axs3[1,0].scatter(t_vec[::200],Q_boundary_tur[::200],label='Flux Turbine',c='g',marker='+')
axs3[1,0].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[1,0].set_ylabel(r'$Q$ [$\mathrm{m}^3/\mathrm{s}$]')
axs3[1,0].legend()

axs3[1,1].set_title('Pressure change vs t=0 at reservoir and turbine')
axs3[1,1].plot(t_vec,pressure_conversion(p_boundary_res-p_boundary_res[0],pUnit_calc, pUnit_conv),label='Reservoir')
axs3[1,1].plot(t_vec,pressure_conversion(p_boundary_tur-p_boundary_tur[0],pUnit_calc, pUnit_conv),label='Turbine')
axs3[1,1].set_xlabel(r'$t$ [$\mathrm{s}$]')
axs3[1,1].set_ylabel(r'$p$ ['+pUnit_conv+']')
axs3[1,1].legend()

fig3.tight_layout()
plt.show()